In [1]:
import pandas as pd

In [2]:
# read_dir = '/work/frink/ramprasad.sa/factuality_metrics/datasets'
# filename = 'aggrefact_llm.csv'
# df = pd.read_csv(f'{read_dir}/{filename}')
# df.head()

In [18]:
def make_data(data_dir,
              genaudit_file,
             annotated_file,
             add_files):
    df_genaudit = pd.read_csv(f'{data_dir}/{genaudit_file}')
    df_annotated = pd.read_csv(f'{data_dir}/{annotated_file}')
    df_summaries = df_genaudit[df_genaudit['id'].isin(df_annotated['docid'])]

    added_dfs = [df_summaries]
    for add_file in add_files:
        df_add = pd.read_csv(f'{data_dir}/{add_file}')
        added_dfs += [df_add]
    return pd.concat(added_dfs)

In [19]:
# df_genaudit = pd.read_csv('/Users/srampras/hallucinations_annotations/scripts/data/genaudit_annotations.csv')
df_annotated = pd.read_csv('/Users/srampras/hallucinations_annotations/scripts/data/test_ann_db_dump_07_12_2024.csv')
# df_genaudit = df_summaries[df_summaries['id'].isin(df_annotated['docid'])]
# df_summaries.head()[:2]

data_dir = '/Users/srampras/hallucinations_annotations/scripts/data'
genaudit_file = 'genaudit_annotations.csv'
annotated_file = 'test_ann_db_dump_07_12_2024.csv'
# add_files = ['llama3_8b_summaries.csv']
add_files = []

df_summaries = make_data(data_dir = data_dir,
                        genaudit_file = genaudit_file,
                        annotated_file = annotated_file,
                        add_files = add_files)

In [6]:
# len(df_genaudit)
# # shortlisted_ids = ['40110402', '39664212', 'REDDIT-9', 'REDDIT-80', 'ACIBENCH-D2N126', 'ACIBENCH-D2N124', 'ACIBENCH-D2N123', 'ACIBENCH-D2N094', 'ACIBENCH-D2N088', 'ACIBENCH-D2N090']
# shortlisted_ids = ['ACIBENCH-D2N106', 'REDDIT-83', '35441162', 'REDDIT-54', 'D2N108' ]
# df_genaudit_sample = []
# for sid in shortlisted_ids:
#     sids = [each for each in df_genaudit['id'] if sid in each]
#     sids = [each for each in sids if 'llama70b' in each or 'gpt4' in each]
#     df_genaudit_sample += [df_genaudit[df_genaudit['id'].isin(sids)]]

# df_genaudit_sample = pd.concat(df_genaudit_sample)
# df_genaudit_sample

In [7]:
import psycopg2

In [8]:
conn = psycopg2.connect(
   database="d65103c5a2kbgs", 
    user='u16ifm924gr2m8', 
    password='pe4029f0ba1fbaafd034f4dbf17f703bf9a5d6c8420f07719f5db4054c9d4da40', 
    host='c1gvrf2q90nbcq.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com', 
    port = '5432'
    
)
cursor = conn.cursor()

In [8]:

# cursor.execute("""DELETE FROM label WHERE user_id='rachel_usher'""")
# # cursor.fetchall()'

In [6]:
conn.commit()

In [9]:


sql_drop = '''DROP TABLE IF EXISTS model_summaries;'''

sql = '''CREATE TABLE model_summaries (
    uuid uuid DEFAULT gen_random_uuid() PRIMARY KEY, 
    docid TEXT NOT NULL ,
    source TEXT,
    summary TEXT NOT NULL,
    model TEXT NOT NULL, 
    benchmark_dataset_name TEXT,
    origin TEXT
);'''

#Creating a database
cursor.execute(sql_drop)
cursor.execute(sql)
print("Table created successfully........")
conn.commit()

Table created successfully........


In [10]:
# cursor = conn.cursor()
# sql_drop = '''DROP TABLE IF EXISTS label;'''
# # d SERIAL PRIMARY KEY,
# #     source_id INT NOT NULL,
# #     start_pos INT NOT NULL,
# #     end_pos INT NOT NULL,
# #     text TEXT NOT NULL,
# #     category VARCHAR(50) NOT NULL,
# #     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP


# create_table_label_str = '''CREATE TABLE label (
#     uuid uuid DEFAULT gen_random_uuid() PRIMARY KEY, 
#     user_id TEXT NOT NULL,
#     docid TEXT NOT NULL,
#     source TEXT NOT NULL,
#     summary TEXT NOT NULL,
#     model TEXT NOT NULL,
#     benchmark_dataset_name TEXT,
#     origin TEXT,
#     nonfactual_span TEXT,
#     error_type TEXT,
#     mistake_severity TEXT,
#     inference_likelihood TEXT,
#     inference_knowledge TEXT,
#     attempt_number INT
# );'''

# cursor.execute(sql_drop)
# cursor.execute(create_table_label_str)
# print("Table created successfully........")
# conn.commit()

In [10]:
cursor = conn.cursor()
sql_drop = '''DROP TABLE IF EXISTS modified_label;'''
# d SERIAL PRIMARY KEY,
#     source_id INT NOT NULL,
#     start_pos INT NOT NULL,
#     end_pos INT NOT NULL,
#     text TEXT NOT NULL,
#     category VARCHAR(50) NOT NULL,
#     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP


create_table_label_str = '''CREATE TABLE modified_label (
    uuid uuid DEFAULT gen_random_uuid() PRIMARY KEY, 
    user_id TEXT NOT NULL,
    docid TEXT NOT NULL,
    source TEXT NOT NULL,
    summary TEXT NOT NULL,
    model TEXT NOT NULL,
    benchmark_dataset_name TEXT,
    origin TEXT,
    nonfactual_span TEXT,
    error_type TEXT,
    mistake_severity TEXT,
    inference_likelihood TEXT,
    inference_knowledge TEXT,
    inference_severity TEXT,
    attempt_number INT
);'''

cursor.execute(sql_drop)
cursor.execute(create_table_label_str)
print("Table created successfully........")
conn.commit()

Table created successfully........


In [11]:
cursor.execute("""SELECT * from modified_label""")
print('Size before adding' , len(cursor.fetchall()))

Size before adding 0


In [12]:
import re 
cursor.execute("""SELECT * from model_summaries""")
print('Size before adding' , len(cursor.fetchall()))

# df_genaudit = df_genaudit[df_genaudit['faithful'] == 1]
df_summaries = df_summaries[~df_summaries['summary'].isnull()]
df_summaries = df_summaries[~df_summaries['source'].isnull()]
for idx, row in df_summaries.iterrows():
    # print(row)
    docid = row['id']
    source = row['source']
    summary = row['summary']
    model = row['model']
    origin = row['origin']
    benchmark_dataset_name = row['benchmark_dataset_name']
    if 'summary' in df_summaries.keys(): 
        # and row['DocID'] in filter_ids:
        summary = row['summary']
        summary_uuid_generic = f'{docid}_{benchmark_dataset_name}_{origin}_{model}'
        summary = re.sub( "'", r"''", summary)
        source = re.sub( "'", r"''", source)
        value_str = f"""'{docid}', '{source}', '{summary}', '{model}', '{benchmark_dataset_name}', '{origin}'"""
        # print(re.findall(r"\'(\w+)\'", dialogue))
        cursor.execute(f"""INSERT INTO model_summaries(docid, source, summary, model, benchmark_dataset_name, origin) VALUES ({value_str})""",
                                                        )
        
   

Size before adding 0


In [13]:
cursor.execute("""SELECT * from model_summaries""")

print('Size after adding' , len(cursor.fetchall()))
conn.commit()

Size after adding 39


In [16]:
cursor.execute("""SELECT * from modified_label""")
cursor.fetchall()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [13]:
# cursor = conn.cursor()
# cursor.execute("""DELETE FROM label WHERE user_id='rachel_usher'""")
# # cursor.fetchall()'